## OCEAN / Personality

In [ ]:
import os 
import pandas as pd
import json
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm

env_path = "/data2/visitor/ASE25/Chimera/.env"
load_dotenv()

# scenario = "Tech-Company"
# company_type_member = "tech_company"
# company_type = "tech"

# scenario = "Finance-Company"
# company_type_member = "finance_corporation"
# company_type = "finance"

scenario = "Medical-Institution"
company_type_member = "medical_institution"
company_type = "medical"

root_dir = "/data2/visitor/ASE25/Chimera"

output_dir = f"{root_dir}/Final-Output/{company_type_member}/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_csv = f'{output_dir}/{company_type_member}_psychometric.csv'

max_attempts = 5

# Reading
log_root_dir = f'/data2/visitor/ASE25/Chimera-Dataset/'

id_role_map = {}
id_list = []
profile_list = []

#######################
profile_output_dir = f"{root_dir}/experiment_output/gemini_{company_type_member}/generated_members"
for file in os.listdir(profile_output_dir):
    if file.endswith(".jsonc"):
        member_profile_path = os.path.join(profile_output_dir, file)
        with open(member_profile_path, 'r') as f:
            member_profile = json.load(f)
        id_role_map[member_profile['id']] = member_profile['role'] # add id-role map
        id_list.append(member_profile['id'])
        profile_list.append(member_profile) # add profile
#######################

# load members.csv
members_csv_path = f"{output_dir}/{company_type_member}_members.csv"
member_df = pd.read_csv(members_csv_path)

for index, row in tqdm(member_df.iterrows(), total=member_df.shape[0], desc="Processing Members"):
    member_id = row['id']
    member_profile = profile_list[id_list.index(member_id)]

    system_prompt = f"""You are an assistant specialized in psychological evaluation using the Big Five personality traits (OCEAN): 
                        Openness, Conscientiousness, Extroversion, Agreeableness, and Neuroticism.\n
                        I will provide you with detailed personal and professional information about an employee in a {scenario}. 
                        Based on this information, please rate the employee on each of the five traits (O, C, E, A, N) on a scale from 1 (lowest) to 40 (highest).\n\n
                        You should only return the five numeric scores representing each index (O, C, E, A, N) in a Python list format.
                        Do not return with anything else."""
    user_prompt = f"""The name of the employee is {member_profile['name']}. The age of the employee is {member_profile['age']}.\n
                        The employee is a {member_profile['role']} in the {company_type} company.\n
                        The MBTI personality type of the employee is {member_profile['mbti']}. The overall information of the employee is {member_profile}.
    """
    for attempt in range(max_attempts):
        client = OpenAI()
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        output = response.choices[0].message.content
        # if response can be convert to a list of 5 integers, break
        try:
            response_list = eval(output)
            if isinstance(response_list, list) and len(response_list) == 5 and all(isinstance(x, int) for x in response_list):
                break
        except Exception as e:
            print("[WARN] Error parsing List:", e, "Retrying...")
            if attempt == max_attempts - 1:
                print("[Error] Max attempts reached. Do not update the schedule.")
    
    # add each value to five columns
    member_df.at[index, 'O'] = int(response_list[0])
    member_df.at[index, 'C'] = int(response_list[1])
    member_df.at[index, 'E'] = int(response_list[2])
    member_df.at[index, 'A'] = int(response_list[3])
    member_df.at[index, 'N'] = int(response_list[4])

# Save the updated DataFrame to a new CSV file
member_df.to_csv(output_csv, index=False)

Processing Members: 100%|██████████| 20/20 [00:18<00:00,  1.08it/s]


In [2]:
member_df

,name,id,role,ip,email,container_id,mbti,interests,personality,O,C,E,A,N
0,Javier Rodriguez,pchmp-1,Physician Champion,10.0.0.19,pc-456789@hospital.org,a1b2c3d4e5f6,ENTJ,"sailing, woodworking, medical history",Highly influential and persuasive. Articulate ...,35.0,37.0,36.0,28.0,16.0
1,Kenji Tanaka,qatest-1,Quality Assurance Tester,10.0.0.26,qa-112233@hospital.org,b8c9d0e1f2g3,INTP,"martial arts (Kendo), PC building, science pod...","Analytical and detail-oriented, with a passion...",35.0,32.0,20.0,28.0,22.0
2,Mei Wong,da-2,Data Analyst,10.0.0.16,da-876543@hospital.org,2b3c4d5e6f7a,INTJ,"astronomy, gourmet cooking, strategy games",Highly analytical and strategic thinker. Enjoy...,35.0,37.0,18.0,22.0,25.0
3,Aaliyah Johnson,rn-3,Registered Nurse,10.0.0.22,rn-432109@hospital.org,9p8o7n6m5l4k,ESTJ,"rock climbing, attending sporting events, volu...","Confident and decisive, with a strong sense of...",30.0,35.0,32.0,28.0,18.0
4,Omar Hassan,dba-1,Database Administrator,10.0.0.17,dba-567890@hospital.org,7e8f9a0b1c2d,ISFJ,"gardening, classic cars, volunteering at the l...",Reliable and detail-oriented. Takes pride in m...,25.0,35.0,15.0,36.0,18.0
5,Priya Sharma,rn-1,Registered Nurse,10.0.0.20,rn-298765@hospital.org,5g6h7i8j9k0l,ISFP,"painting, yoga, trying new ethnic foods","Compassionate and empathetic, with a natural a...",30.0,34.0,18.0,36.0,22.0
6,Ricardo Alvarez,itpm-1,IT Project Manager,10.0.0.10,pm-781254@hospital.org,92b3da6f21e8,ESTJ,"Golf, wood-working, historical documentaries",Highly organized and detail-oriented. A natura...,25.0,36.0,28.0,27.0,18.0
7,Anika Sharma,helpdesk-1,Help Desk Support,10.0.0.29,help-192837@hospital.org,1b2c3d4e5f6a,INFP,"reading fantasy novels, playing ukulele, volun...","Empathetic and patient, with a genuine desire ...",32.0,26.0,18.0,35.0,28.0
8,Liam OConnell,rn-2,Registered Nurse,10.0.0.21,rn-765432@hospital.org,1z2y3x4w5v6u,ESFP,"live music, hiking, craft beer","Outgoing and enthusiastic, with a talent for m...",32.0,35.0,37.0,36.0,15.0
9,Carlos Ramirez,da-1,Data Analyst,10.0.0.15,da-345678@hospital.org,89a7b6c5d4e3,ESTP,"competitive cycling, craft breweries, live music",Outgoing and results-oriented. Enjoys working ...,26.0,35.0,37.0,30.0,20.0
